<a href="https://colab.research.google.com/github/himanshu530/Complete-Deep-Learning/blob/master/cnn_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#creating cnn via tranfer learning keras api

In [2]:
from keras.layers import Input,Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import  preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator  #Data Augmentation
from keras.models import Sequential
import numpy as np
from glob import glob

import matplotlib.pyplot as plt


In [3]:
IMAGE_SIZE = [224,224]

In [4]:
train_path = 'Datasets/train'
test_path = 'Datasets/test'

In [5]:
#Adding preprocessing layer to the front of vgg
vgg = VGG16(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)  #+3 to add rgb color channels

58892288/58889256 [==============================] - 2s 0us/step


In [6]:
#Dont train the existing weights

for layer in vgg.layers:
  layer.trainable = False

In [8]:
folders = glob('Datasets/Train/*')  #To get the number of output classes

In [9]:
x = Flatten()(vgg.output)

In [10]:
prediction = Dense(len(folders),activation='softmax')(x)

In [11]:
model = Model(inputs=vgg.input,outputs=prediction)

In [12]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [13]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)

In [14]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Data Augmentation to increase the train and test data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('Datasets/train',target_size = (224,224),batch_size=32,class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/test',target_size=(224,224),batch_size = 32,class_mode = 'categorical')



r = model.fit_generator(training_set,validation_data = test_set,epochs = 5,steps_per_epoch=len(training_set),validation_steps=len(test_set))


In [ ]:
## predictions here